<b>K-fold Cross Validation Model</b>

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
DATADIR = "DATASET"
CATEDIR = ["White","Red","Green","Blue","Black"]

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

In [4]:
training_data = []
IMG_SIZE = 100

def create_training_data():
    for class_folder in CATEDIR:
        path = DATADIR+"/"+class_folder
        # give an index to each class folder
        class_index = CATEDIR.index(class_folder)
        for img in os.listdir(path):
            img_array = cv2.imread(path+"/"+img)
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE), 3)
            training_data.append([new_array, class_index])
create_training_data()

import random
random.shuffle(training_data)

X = []
y = []

for images, classes in training_data:
    X.append(images)
    y.append(classes)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y)

X_cross = X / 255.0
y_cross = to_categorical(y, 5)

In [5]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [6]:
def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape = (IMG_SIZE, IMG_SIZE, 3), 
                     activation = "relu"))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(32, (3, 3), activation = "relu"))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(64, (3, 3), activation = "relu"))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(64, (3, 3), activation = "relu"))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(128, (3, 3), activation = "relu"))
    model.add(MaxPooling2D(pool_size = (2, 2)))

    model.add(Flatten())

    model.add(Dense(256, activation = "relu"))
    model.add(Dense(256, activation = "relu"))
    model.add(Dense(len(CATEDIR), activation = "softmax"))

    model.compile(loss = "categorical_crossentropy", 
                  optimizer = "adam", 
                  metrics = ["accuracy"])
    return model

In [7]:
model_cross = KerasClassifier(build_fn = build_model, epochs = 10, verbose = 0)

from sklearn.model_selection import cross_val_score, KFold

cv = KFold(3, shuffle = True)
scores = cross_val_score(model_cross, X_cross, y_cross, cv = cv)

scores

array([0.88450146, 0.85587364, 0.89822137])

In [8]:
print("Accuracy {:0.4f} ~ {:0.4f}" .format(scores.mean(), scores.std()))

Accuracy 0.8795 ~ 0.0176
